In [1]:
# %pip install sqlalchemy
%rm tutorial.db

In [2]:
import sqlalchemy

print(sqlalchemy.__version__)

2.0.25


In [3]:
from sqlalchemy import create_engine

engine = create_engine("sqlite+pysqlite:///tutorial.db", echo=True)

In [4]:
from sqlalchemy import text

# "commit as you go"

with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}],
    )
    conn.commit()

# "beging once"

with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}],
    )

# Session commiting, statement objects

from sqlalchemy.orm import Session

stmt = text("INSERT INTO some_table (x, y) VALUES (:x, :y)")
with Session(engine) as session:
    session.execute(stmt, 
                    [{"x": 11, "y": 12}, {"x": 13, "y": 14}])
    session.commit()

2024-02-12 16:34:19,199 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-12 16:34:19,205 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2024-02-12 16:34:19,213 INFO sqlalchemy.engine.Engine [generated in 0.01360s] ()
2024-02-12 16:34:19,257 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2024-02-12 16:34:19,261 INFO sqlalchemy.engine.Engine [generated in 0.00414s] [(1, 1), (2, 4)]
2024-02-12 16:34:19,273 INFO sqlalchemy.engine.Engine COMMIT
2024-02-12 16:34:19,284 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-12 16:34:19,292 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2024-02-12 16:34:19,295 INFO sqlalchemy.engine.Engine [cached since 0.03857s ago] [(6, 8), (9, 10)]
2024-02-12 16:34:19,301 INFO sqlalchemy.engine.Engine COMMIT
2024-02-12 16:34:20,035 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-12 16:34:20,038 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
20

In [5]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

In [6]:
from sqlalchemy import Table, Column, Integer, String

user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

In [7]:
from sqlalchemy import ForeignKey

address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False),
)

In [8]:
metadata_obj.create_all(engine)

2024-02-12 16:34:20,232 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-12 16:34:20,239 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2024-02-12 16:34:20,244 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-12 16:34:20,249 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2024-02-12 16:34:20,277 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-12 16:34:20,290 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2024-02-12 16:34:20,294 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-12 16:34:20,299 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2024-02-12 16:34:20,302 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-12 16:34:20,309 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2024-02-12 16:34:20,312 INFO sqlalchemy.engine.Engine [no key 0.00325s] ()
2024-02-12 16:34:20,331 INFO sqlalchemy.engine.Engine 
C

In [9]:
from sqlalchemy.orm import DeclarativeBase

class Base(DeclarativeBase):
    pass

In [10]:
from typing import List, Optional
from sqlalchemy.orm import Mapped, mapped_column, relationship

class User(Base):
    __tablename__ = "user_account"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]

    addresses: Mapped[List["Address"]] = relationship(back_populates="user")

    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

In [11]:
from sqlalchemy import ForeignKey

class Address(Base):
    __tablename__ = "address"

    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id = mapped_column(ForeignKey("user_account.id"))

    user: Mapped[User] = relationship(back_populates="addresses")

    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [12]:
Base.metadata.create_all(engine)

2024-02-12 16:34:20,564 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-12 16:34:20,579 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2024-02-12 16:34:20,586 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-12 16:34:20,593 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2024-02-12 16:34:20,597 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-12 16:34:20,605 INFO sqlalchemy.engine.Engine COMMIT


In [13]:
Table("some_table", metadata_obj, autoload_with=engine)

2024-02-12 16:34:20,649 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-12 16:34:20,653 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("some_table")
2024-02-12 16:34:20,657 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-12 16:34:20,670 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type in ('table', 'view')
2024-02-12 16:34:20,673 INFO sqlalchemy.engine.Engine [raw sql] ('some_table',)
2024-02-12 16:34:20,685 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("some_table")
2024-02-12 16:34:20,688 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-12 16:34:20,696 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("some_table")
2024-02-12 16:34:20,699 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-12 16:34:20,704 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type i

Table('some_table', MetaData(), Column('x', INTEGER(), table=<some_table>), Column('y', INTEGER(), table=<some_table>), schema=None)

In [14]:
from sqlalchemy import insert

# Automatic insert statement
print(insert(user_table))

INSERT INTO user_account (id, name, fullname) VALUES (:id, :name, :fullname)


In [15]:
# Insert statement for specific fields with values method
stmt = insert(user_table).values(name='mordy', fullname='Mordechai Fast')

compiled = stmt.compile()
compiled.params

{'name': 'mordy', 'fullname': 'Mordechai Fast'}

In [16]:
with engine.begin() as conn:
    result = conn.execute(stmt)
    # Notice shorter SQL text, with '?' and no 'id'

2024-02-12 16:34:21,104 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-12 16:34:21,187 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2024-02-12 16:34:21,192 INFO sqlalchemy.engine.Engine [generated in 0.01237s] ('mordy', 'Mordechai Fast')
2024-02-12 16:34:21,201 INFO sqlalchemy.engine.Engine COMMIT


In [17]:
# Primamry key is created automatically
result.inserted_primary_key

(1,)

In [18]:
# Insertions can be specified in the execute statement
with engine.begin() as conn:
    result = conn.execute(insert(user_table),
                          [{"name": "golda", "fullname": "Golda Fast"},
                           {"name": "etty", "fullname": "Esther Fast"}])

2024-02-12 16:34:21,280 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-12 16:34:21,290 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2024-02-12 16:34:21,308 INFO sqlalchemy.engine.Engine [generated in 0.01837s] [('golda', 'Golda Fast'), ('etty', 'Esther Fast')]
2024-02-12 16:34:21,317 INFO sqlalchemy.engine.Engine COMMIT


In [19]:
# Unneccessarally complicated, core-only method
from sqlalchemy import select, bindparam

scalar_subq = (
    select(user_table.c.id)
    .where(user_table.c.name == bindparam("username"))
    .scalar_subquery()
)

with engine.begin() as conn:
    result = conn.execute(
        insert(address_table).values(user_id=scalar_subq),
        [
            {"username": "mordy", "email_address": "mordechai.fast@gmail.com"},
            {"username": "golda", "email_address": "golda.fast@gmail.com"},
            {"username": "golda", "email_address": "melechmommy@gmail.com"},
        ],
    )

2024-02-12 16:34:21,367 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-12 16:34:21,396 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) VALUES ((SELECT user_account.id 
FROM user_account 
WHERE user_account.name = ?), ?)
2024-02-12 16:34:21,398 INFO sqlalchemy.engine.Engine [generated in 0.00432s] [('mordy', 'mordechai.fast@gmail.com'), ('golda', 'golda.fast@gmail.com'), ('golda', 'melechmommy@gmail.com')]
2024-02-12 16:34:21,408 INFO sqlalchemy.engine.Engine COMMIT


In [20]:
insert_return_stmt = insert(address_table).returning(
    address_table.c.id, address_table.c.email_address
)

with engine.connect() as conn:
    result = conn.execute(insert_return_stmt,
        [{"user_id": 1, "email_address": "none@no.where"}])
    # Don't save that!
print(f"The result is: {result.first()}")

2024-02-12 16:34:21,453 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-12 16:34:21,458 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) VALUES (?, ?) RETURNING id, email_address
2024-02-12 16:34:21,463 INFO sqlalchemy.engine.Engine [generated in 0.01007s] (1, 'none@no.where')
2024-02-12 16:34:21,472 INFO sqlalchemy.engine.Engine ROLLBACK
The result is: (4, 'none@no.where')


In [21]:
select_stmt = select(user_table.c.id, user_table.c.name + "@aol.com")
insert_from_stmt = insert(address_table).from_select(
    ["user_id", "email_address"], select_stmt
)

with engine.connect() as conn:
    result = conn.execute(insert_from_stmt.returning(
        address_table.c.id, address_table.c.email_address))
    print(*result, sep="\n")
    conn.commit()

2024-02-12 16:34:21,540 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-12 16:34:21,553 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || ? AS anon_1 
FROM user_account RETURNING id, email_address
2024-02-12 16:34:21,559 INFO sqlalchemy.engine.Engine [generated in 0.02009s] ('@aol.com',)
(4, 'mordy@aol.com')
(5, 'golda@aol.com')
(6, 'etty@aol.com')
2024-02-12 16:34:21,572 INFO sqlalchemy.engine.Engine COMMIT


In [22]:
from sqlalchemy import select

where_stmt = select(user_table).where(user_table.c.name == "mordy")

with engine.connect() as conn:
    result = conn.execute(where_stmt)
print(*result)

2024-02-12 16:34:21,637 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-12 16:34:21,642 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2024-02-12 16:34:21,646 INFO sqlalchemy.engine.Engine [generated in 0.00882s] ('mordy',)
2024-02-12 16:34:21,650 INFO sqlalchemy.engine.Engine ROLLBACK
(1, 'mordy', 'Mordechai Fast')


In [23]:
orm_where_stmt = select(User).where(User.name == "mordy")

with Session(engine) as session:
    result = session.execute(orm_where_stmt)
    # Must be used before sesssion expires
    print(*result)

2024-02-12 16:34:21,699 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-12 16:34:21,750 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2024-02-12 16:34:21,753 INFO sqlalchemy.engine.Engine [generated in 0.00351s] ('mordy',)
(User(id=1, name='mordy', fullname='Mordechai Fast'),)
2024-02-12 16:34:21,761 INFO sqlalchemy.engine.Engine ROLLBACK


In [24]:
# Two ways to refer to the column names
partial_select1 = select(user_table.c.name, user_table.c.fullname)
partial_select2 = select(user_table.c["name", "fullname"])
print(partial_select1, partial_select2, sep='\n')

SELECT user_account.name, user_account.fullname 
FROM user_account
SELECT user_account.name, user_account.fullname 
FROM user_account


In [25]:
session.execute(select(User)).all()
# Notice the User objects are returned in single-tuples

2024-02-12 16:34:21,937 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-12 16:34:21,948 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2024-02-12 16:34:21,958 INFO sqlalchemy.engine.Engine [generated in 0.00959s] ()


[(User(id=1, name='mordy', fullname='Mordechai Fast'),),
 (User(id=2, name='golda', fullname='Golda Fast'),),
 (User(id=3, name='etty', fullname='Esther Fast'),)]

In [26]:
session.scalars(select(User)).all()

2024-02-12 16:34:22,013 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2024-02-12 16:34:22,017 INFO sqlalchemy.engine.Engine [cached since 0.06883s ago] ()


[User(id=1, name='mordy', fullname='Mordechai Fast'),
 User(id=2, name='golda', fullname='Golda Fast'),
 User(id=3, name='etty', fullname='Esther Fast')]

In [27]:
session.scalar(select(User))
# Returns the first row as an object

2024-02-12 16:34:22,058 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2024-02-12 16:34:22,063 INFO sqlalchemy.engine.Engine [cached since 0.115s ago] ()


User(id=1, name='mordy', fullname='Mordechai Fast')

In [28]:
# Usefull when searching for a primary key
session.scalar(select(User).where(User.id == 2))

2024-02-12 16:34:22,181 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.id = ?
2024-02-12 16:34:22,191 INFO sqlalchemy.engine.Engine [generated in 0.01113s] (2,)


User(id=2, name='golda', fullname='Golda Fast')

In [29]:
# Selecting specific columns
session.execute(select(User.name, User.fullname)).all()

2024-02-12 16:34:22,238 INFO sqlalchemy.engine.Engine SELECT user_account.name, user_account.fullname 
FROM user_account
2024-02-12 16:34:22,242 INFO sqlalchemy.engine.Engine [generated in 0.00444s] ()


[('mordy', 'Mordechai Fast'), ('golda', 'Golda Fast'), ('etty', 'Esther Fast')]

In [30]:
session.execute(
    select(User.name, Address).where(User.id == Address.user_id)
).all()

2024-02-12 16:34:22,314 INFO sqlalchemy.engine.Engine SELECT user_account.name, address.id, address.email_address, address.user_id 
FROM user_account, address 
WHERE user_account.id = address.user_id
2024-02-12 16:34:22,319 INFO sqlalchemy.engine.Engine [generated in 0.00500s] ()


[('mordy', Address(id=1, email_address='mordechai.fast@gmail.com')),
 ('golda', Address(id=2, email_address='golda.fast@gmail.com')),
 ('golda', Address(id=3, email_address='melechmommy@gmail.com')),
 ('mordy', Address(id=4, email_address='mordy@aol.com')),
 ('golda', Address(id=5, email_address='golda@aol.com')),
 ('etty', Address(id=6, email_address='etty@aol.com'))]

In [31]:
label_selection = select(
    ("Username: " + user_table.c.name).label("username")
).order_by(user_table.c.name)

with engine.connect() as conn:
    for row in conn.execute(label_selection):
        print(row.username)

2024-02-12 16:34:22,386 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-12 16:34:22,391 INFO sqlalchemy.engine.Engine SELECT ? || user_account.name AS username 
FROM user_account ORDER BY user_account.name
2024-02-12 16:34:22,397 INFO sqlalchemy.engine.Engine [generated in 0.01182s] ('Username: ',)
Username: etty
Username: golda
Username: mordy
2024-02-12 16:34:22,407 INFO sqlalchemy.engine.Engine ROLLBACK


In [32]:
session.execute(select(text("'SQL'"), user_table.c.fullname)).all()

2024-02-12 16:34:22,437 INFO sqlalchemy.engine.Engine SELECT 'SQL', user_account.fullname 
FROM user_account
2024-02-12 16:34:22,449 INFO sqlalchemy.engine.Engine [generated in 0.01242s] ()


[('SQL', 'Mordechai Fast'), ('SQL', 'Golda Fast'), ('SQL', 'Esther Fast')]

In [33]:
from sqlalchemy import literal_column

lit_col_stmt = select(
    literal_column("'Name'").label("desc"), user_table.c.fullname
)
with engine.connect() as conn:
    for row in conn.execute(lit_col_stmt):
        print(f"{row.desc}: {row.fullname}")

2024-02-12 16:34:22,510 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-12 16:34:22,515 INFO sqlalchemy.engine.Engine SELECT 'Name' AS "desc", user_account.fullname 
FROM user_account
2024-02-12 16:34:22,521 INFO sqlalchemy.engine.Engine [generated in 0.01095s] ()
Name: Mordechai Fast
Name: Golda Fast
Name: Esther Fast
2024-02-12 16:34:22,529 INFO sqlalchemy.engine.Engine ROLLBACK


In [34]:
multi_where1 = (
    select(address_table)
    .where(user_table.c.name == 'golda')
    .where(address_table.c.user_id == user_table.c.id)
)

multi_where2 = select(address_table).where(
    user_table.c.name == 'golda',
    address_table.c.user_id == user_table.c.id,
)

print(multi_where1, multi_where2, sep='\n')

SELECT address.id, address.user_id, address.email_address 
FROM address, user_account 
WHERE user_account.name = :name_1 AND address.user_id = user_account.id
SELECT address.id, address.user_id, address.email_address 
FROM address, user_account 
WHERE user_account.name = :name_1 AND address.user_id = user_account.id


In [35]:
from sqlalchemy import and_, or_, not_

compound_stmt1 = select(Address.email_address).where(or_(
    and_(User.name == 'golda', User.id == Address.user_id),
    not_(Address.user_id == 0)
))

compound_stmt2 = select(Address.email_address).where(
    (User.name == 'golda') & (User.id == Address.user_id)
    | ~(Address.user_id == 0)
)

print(compound_stmt1, compound_stmt2, sep='\n')

SELECT address.email_address 
FROM address, user_account 
WHERE user_account.name = :name_1 AND user_account.id = address.user_id OR address.user_id != :user_id_1
SELECT address.email_address 
FROM address, user_account 
WHERE user_account.name = :name_1 AND user_account.id = address.user_id OR address.user_id != :user_id_1


In [36]:
join_stmt1 = (
    select(user_table.c.name, address_table.c.email_address)
    .join_from(user_table, address_table)
)

join_stmt2 = (
    select(user_table.c.name, address_table.c.email_address)
    .join(address_table)
)

print(join_stmt1, join_stmt2, sep='\n')
# ON condition is inferred

SELECT user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id
SELECT user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


In [37]:
from sqlalchemy import func

count_overlap = (select(func.count('*'))
                 .select_from(user_table).join(address_table))

with engine.connect() as conn:
    count = conn.execute(count_overlap)
print(*count)

2024-02-12 16:34:22,765 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-12 16:34:22,769 INFO sqlalchemy.engine.Engine SELECT count(?) AS count_1 
FROM user_account JOIN address ON user_account.id = address.user_id
2024-02-12 16:34:22,776 INFO sqlalchemy.engine.Engine [generated in 0.01363s] ('*',)
2024-02-12 16:34:22,782 INFO sqlalchemy.engine.Engine ROLLBACK
(6,)


In [38]:
# SQL - LEFT OUTER JOIN, FULL OUTER JOIN
print(select(user_table).join(address_table, isouter=True))
print(select(user_table).outerjoin(address_table))
print(select(user_table).join(address_table, full=True))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account LEFT OUTER JOIN address ON user_account.id = address.user_id
SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account LEFT OUTER JOIN address ON user_account.id = address.user_id
SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account FULL OUTER JOIN address ON user_account.id = address.user_id


In [39]:
with engine.connect() as conn:
    result = conn.execute(select(user_table).order_by(user_table.c.name))
    # equivilent to .order_by(user_table.c.name.asc())
print(*result, sep='\n')

2024-02-12 16:34:22,877 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-12 16:34:22,881 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account ORDER BY user_account.name
2024-02-12 16:34:22,885 INFO sqlalchemy.engine.Engine [generated in 0.00865s] ()
2024-02-12 16:34:22,890 INFO sqlalchemy.engine.Engine ROLLBACK
(3, 'etty', 'Esther Fast')
(2, 'golda', 'Golda Fast')
(1, 'mordy', 'Mordechai Fast')


In [40]:
with engine.connect() as conn:
    result = conn.execute(select(address_table)
                          .order_by(address_table.c.id.desc()))
print(*result, sep='\n')

2024-02-12 16:34:22,943 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-12 16:34:22,946 INFO sqlalchemy.engine.Engine SELECT address.id, address.user_id, address.email_address 
FROM address ORDER BY address.id DESC
2024-02-12 16:34:22,950 INFO sqlalchemy.engine.Engine [generated in 0.00775s] ()
2024-02-12 16:34:22,962 INFO sqlalchemy.engine.Engine ROLLBACK
(6, 3, 'etty@aol.com')
(5, 2, 'golda@aol.com')
(4, 1, 'mordy@aol.com')
(3, 2, 'melechmommy@gmail.com')
(2, 2, 'golda.fast@gmail.com')
(1, 1, 'mordechai.fast@gmail.com')


In [41]:
with engine.connect() as conn:
    result = conn.execute(
        select(User.name, func.count(Address.id))
        .join(Address)
        .group_by(User.name)
        .having(func.count(Address.id) > 1)
    )
print(*result, sep=', ')

2024-02-12 16:34:23,025 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-12 16:34:23,031 INFO sqlalchemy.engine.Engine SELECT user_account.name, count(address.id) AS count_1 
FROM user_account JOIN address ON user_account.id = address.user_id GROUP BY user_account.name 
HAVING count(address.id) > ?
2024-02-12 16:34:23,035 INFO sqlalchemy.engine.Engine [generated in 0.00963s] (1,)
2024-02-12 16:34:23,041 INFO sqlalchemy.engine.Engine ROLLBACK
('golda', 3), ('mordy', 2)


In [42]:
from sqlalchemy import desc

with engine.connect() as conn:
    result = conn.execute(
        select(User.name, func.count(Address.id).label("num_addresses"))
        .join(Address)
        .group_by("name")
        .order_by(desc("num_addresses"))
    )
print(*result, sep='\n')

2024-02-12 16:34:23,105 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-12 16:34:23,110 INFO sqlalchemy.engine.Engine SELECT user_account.name, count(address.id) AS num_addresses 
FROM user_account JOIN address ON user_account.id = address.user_id GROUP BY user_account.name ORDER BY num_addresses DESC
2024-02-12 16:34:23,114 INFO sqlalchemy.engine.Engine [generated in 0.00928s] ()
2024-02-12 16:34:23,120 INFO sqlalchemy.engine.Engine ROLLBACK
('golda', 3)
('mordy', 2)
('etty', 1)


In [43]:
user1 = user_table.alias()
user2 = user_table.alias()

with engine.connect() as conn:
    result = conn.execute(
        select(user1.c.name, user2.c.name)
        .join_from(user1, user2, user1.c.id < user2.c.id)
    )
print(*result, sep=', ')

2024-02-12 16:34:23,159 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-12 16:34:23,171 INFO sqlalchemy.engine.Engine SELECT user_account_1.name, user_account_2.name AS name_1 
FROM user_account AS user_account_1 JOIN user_account AS user_account_2 ON user_account_1.id < user_account_2.id
2024-02-12 16:34:23,190 INFO sqlalchemy.engine.Engine [generated in 0.03084s] ()
2024-02-12 16:34:23,198 INFO sqlalchemy.engine.Engine ROLLBACK
('mordy', 'golda'), ('mordy', 'etty'), ('golda', 'etty')


In [44]:
from sqlalchemy.orm import aliased

address1 = aliased(Address)
address2 = aliased(Address)

with Session(engine) as session:
    result = session.execute(
        select(User)
        .join(address1)
        .where(address1.email_address == "golda.fast@gmail.com")
        .join(address2)
        .where(address2.email_address == "melechmommy@gmail.com")
    )
    print(*result, sep=', ')

2024-02-12 16:34:23,303 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-12 16:34:23,326 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account JOIN address AS address_1 ON user_account.id = address_1.user_id JOIN address AS address_2 ON user_account.id = address_2.user_id 
WHERE address_1.email_address = ? AND address_2.email_address = ?
2024-02-12 16:34:23,340 INFO sqlalchemy.engine.Engine [generated in 0.01516s] ('golda.fast@gmail.com', 'melechmommy@gmail.com')
(User(id=2, name='golda', fullname='Golda Fast'),)
2024-02-12 16:34:23,350 INFO sqlalchemy.engine.Engine ROLLBACK


In [45]:
subq = (
    select(func.count(address_table.c.id).label("count"),
           address_table.c.user_id)
    .group_by(address_table.c.user_id)
    .subquery()
)

with engine.connect() as conn:
    result = conn.execute(select(user_table.c.name, subq.c.count)
                          .join(user_table))
print(*result, sep=', ')

2024-02-12 16:34:23,471 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-12 16:34:23,476 INFO sqlalchemy.engine.Engine SELECT user_account.name, anon_1.count 
FROM (SELECT count(address.id) AS count, address.user_id AS user_id 
FROM address GROUP BY address.user_id) AS anon_1 JOIN user_account ON user_account.id = anon_1.user_id
2024-02-12 16:34:23,483 INFO sqlalchemy.engine.Engine [generated in 0.01330s] ()
2024-02-12 16:34:23,488 INFO sqlalchemy.engine.Engine ROLLBACK
('mordy', 2), ('golda', 3), ('etty', 1)


In [46]:
cte = (
    select(func.count(address_table.c.id).label("count"),
           address_table.c.user_id)
    .group_by(address_table.c.user_id)
    .cte("raw_answer")
)

with engine.connect() as conn:
    result = conn.execute(select(user_table.c.name, cte.c.count)
                          .join(user_table))
print(*result, sep=', ')

2024-02-12 16:34:23,558 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-12 16:34:23,561 INFO sqlalchemy.engine.Engine WITH raw_answer AS 
(SELECT count(address.id) AS count, address.user_id AS user_id 
FROM address GROUP BY address.user_id)
 SELECT user_account.name, raw_answer.count 
FROM raw_answer JOIN user_account ON user_account.id = raw_answer.user_id
2024-02-12 16:34:23,565 INFO sqlalchemy.engine.Engine [generated in 0.00775s] ()
2024-02-12 16:34:23,572 INFO sqlalchemy.engine.Engine ROLLBACK
('mordy', 2), ('golda', 3), ('etty', 1)


In [47]:
subq = (select(Address).where(~Address.email_address.like("%fast@gmail.com"))
        .subquery("interesting_mail")) # or cte()
address_sq = aliased(Address, subq)
stmt = (
    select(User.name, address_sq.email_address)
    .join_from(User, address_sq)
    .order_by(User.id)
)

with Session(engine) as session:
    for user, address in session.execute(stmt):
        print(user, address)

2024-02-12 16:34:23,627 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-12 16:34:23,641 INFO sqlalchemy.engine.Engine SELECT user_account.name, interesting_mail.email_address 
FROM user_account JOIN (SELECT address.id AS id, address.email_address AS email_address, address.user_id AS user_id 
FROM address 
WHERE address.email_address NOT LIKE ?) AS interesting_mail ON user_account.id = interesting_mail.user_id ORDER BY user_account.id
2024-02-12 16:34:23,645 INFO sqlalchemy.engine.Engine [generated in 0.00471s] ('%fast@gmail.com',)
mordy mordy@aol.com
golda melechmommy@gmail.com
golda golda@aol.com
etty etty@aol.com
2024-02-12 16:34:23,655 INFO sqlalchemy.engine.Engine ROLLBACK


In [52]:
subq = (
    select(func.count(address_table.c.id))
    .where(user_table.c.id == address_table.c.user_id)
    .subquery()
)
scalar = (
    select(func.count(address_table.c.id))
    .where(user_table.c.id == address_table.c.user_id)
    .scalar_subquery()
)
print(subq, scalar, subq == 3, scalar == 3, sep='\n*** ')

SELECT count(address.id) AS count_1 
FROM address, user_account 
WHERE user_account.id = address.user_id
*** (SELECT count(address.id) AS count_1 
FROM address, user_account 
WHERE user_account.id = address.user_id)
*** False
*** (SELECT count(address.id) AS count_1 
FROM address, user_account 
WHERE user_account.id = address.user_id) = :param_1


In [52]:
# correlation
print(select(user_table.c.name, scalar))

SELECT user_account.name, (SELECT count(address.id) AS count_1 
FROM address 
WHERE user_account.id = address.user_id) AS anon_1 
FROM user_account


In [53]:
# Problematic correlation
stmt = (
    select(
        user_table.c.name,
        address_table.c.email_address,
        scalar.label("address_count"),
    )
    .join_from(user_table, address_table)
    .order_by(user_table.c.id, address_table.c.id)
)
print(stmt)

InvalidRequestError: Select statement '<sqlalchemy.sql.selectable.Select object at 0x7f355ee07a40>' returned no FROM clauses due to auto-correlation; specify correlate(<tables>) to control correlation manually.

In [55]:
correlated = scalar.correlate(user_table)
print(correlated)

(SELECT count(address.id) AS count_1 
FROM address, user_account 
WHERE user_account.id = address.user_id)


In [56]:
stmt = (
    select(
        user_table.c.name,
        address_table.c.email_address,
        correlated.label("address_count"),
    )
    .join_from(user_table, address_table)
    .order_by(user_table.c.id, address_table.c.id)
)

with engine.connect() as conn:
    result = conn.execute(stmt)
print(*result, sep='\n')

2024-02-14 00:36:38,663 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-14 00:36:38,665 INFO sqlalchemy.engine.Engine SELECT user_account.name, address.email_address, (SELECT count(address.id) AS count_1 
FROM address 
WHERE user_account.id = address.user_id) AS address_count 
FROM user_account JOIN address ON user_account.id = address.user_id ORDER BY user_account.id, address.id
2024-02-14 00:36:38,665 INFO sqlalchemy.engine.Engine [generated in 0.00208s] ()
2024-02-14 00:36:38,666 INFO sqlalchemy.engine.Engine ROLLBACK
('mordy', 'mordechai.fast@gmail.com', 2)
('mordy', 'mordy@aol.com', 2)
('golda', 'golda.fast@gmail.com', 3)
('golda', 'melechmommy@gmail.com', 3)
('golda', 'golda@aol.com', 3)
('etty', 'etty@aol.com', 1)
